In [1]:
import os

In [2]:
%pwd

'/Users/j_kanishkha/Kidney_Disease_Classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/j_kanishkha/Kidney_Disease_Classification'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Jkanishkha0305/Kidney_Disease_Classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Jkanishkha0305"
os.environ["MLFLOW_TRACKING_PASSWORD"]="6bd16d7f5ee713eba8329fb353c637dc8de93b55"

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from kidneyClassifier.constants import *
from kidneyClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/Jkanishkha0305/Kidney_Disease_Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [18]:
import dagshub
dagshub.init(repo_owner='Jkanishkha0305', repo_name='Kidney_Disease_Classification', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

/opt/anaconda3/envs/kidney/lib/python3.8/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=21cea0e5-7e8d-4e05-abea-066dc205cfe6&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=bcc417aa6cc1effc7f38283d7948f3046c1fef62510872b4d3a818c49d6c5ff2


[2024-10-21 14:49:01,051: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/middleman "HTTP/1.1 200 OK"]


[2024-10-21 14:49:01,191: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/access_token "HTTP/1.1 200 OK"]
[2024-10-21 14:49:01,277: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as Jkanishkha0305

[2024-10-21 14:49:01,282: INFO: helpers: Accessing as Jkanishkha0305]
[2024-10-21 14:49:01,465: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/Jkanishkha0305/Kidney_Disease_Classification "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "Jkanishkha0305/Kidney_Disease_Classification"

[2024-10-21 14:49:01,475: INFO: helpers: Initialized MLflow to track repo "Jkanishkha0305/Kidney_Disease_Classification"]


Repository Jkanishkha0305/Kidney_Disease_Classification initialized!

[2024-10-21 14:49:01,477: INFO: helpers: Repository Jkanishkha0305/Kidney_Disease_Classification initialized!]


In [15]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [17]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-10-21 15:09:26,525: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-10-21 15:09:26,528: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-21 15:09:26,529: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 10s 1s/step - loss: 1.2637 - accuracy: 0.6187
[2024-10-21 15:09:36,930: INFO: common: json file saved at: scores.json]


2024/10/21 15:09:37 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/js/5xf1p1214jl9hwnj5cj67gzm0000gn/T/tmp9xefx9lt/model/data/model/assets
[2024-10-21 15:09:37,857: INFO: builder_impl: Assets written to: /var/folders/js/5xf1p1214jl9hwnj5cj67gzm0000gn/T/tmp9xefx9lt/model/data/model/assets]


2024/10/21 15:09:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/10/21 15:09:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.
2024/10/21 15:09:57 INFO mlflow.tracking._tracking_service.client: 🏃 View run calm-ox-765 at: https://dagshub.com/Jkanishkha0305/Kidney_Disease_Classification.mlflow/#/experiments/0/runs/e0ce35a7d18940d1948e9159015e761c.
2024/10/21 15:09:57 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Jkanishkha0305/Kidney_Disease_Classification.mlflow/#/experiments/0.


In [21]:
pip show tensorflow keras

Name: tensorflow
Version: 2.13.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /opt/anaconda3/envs/kidney/lib/python3.8/site-packages
Requires: tensorflow-macos
Required-by: 
---
Name: keras
Version: 2.13.1
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: /opt/anaconda3/envs/kidney/lib/python3.8/site-packages
Requires: 
Required-by: tensorflow-macos
Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install --upgrade mlflow

  Using cached opentelemetry_api-1.27.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_sdk-1.27.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached graphql_relay-3.2.0-py3-none-any.whl.metadata (12 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached opentelemetry_semantic_conventions-0.48b0-py3-none-any.whl.metadata (2.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 38.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.4/568.4 kB 42.6 MB/s eta 0:00:00
Using cached graphql_relay-3.2.0-py3-none-any.whl (16 kB)
Using cached opentelemetry_api-1.27.0-py3-none-any.whl (63 kB)
Using cached opentelemetry_sdk-1.27.0-py3-none-any.whl (110 kB)
Using cached opentelemetry_semantic_conventions-0.48b0-py3-none-any.whl (149 kB)
Using cached Deprecated-1.2.14-py2.py3-none-any.whl (9.6 kB)
  Attempting uninstall: mlflow
    Found exi